# Group Exercise 6 - Corner flow

This script plots streamlines and potential lines for the "corner-flow with air blown through a slit" group exercise. It results from the superposition of a source flow and a wedge flow.

## Preparation

To prepare the plot, several helper functions have to be defined. This is done in the following cell.
First, the two libraries numpy and matplotlib.pyplot have to be imported in order to user their functionality. Note, these libraries are helpful in many scenarios, when data should be plotted. Next, helper classes that provide the complex velocity for the source-/sink-flow and the parallel flow are implemented. Finally, a function providing the meshgrid for the calculations is implemented.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define important colors
TUMOrange = "#E37222"
TUMBlue = "#3070b3"

# Define helper classes to get the complex velocity of elementary flows
class WedgeFlow:

    def __init__(self, wedge_angle: float, c_factor: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the source- and sink-flow class. Stores relevant quantities as class members.

        Args:
            wedge_angle (float): The wedge angle in degrees.
            c_factor (float): The constant C for the complex potential of the wedge flow. 
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y

        wedge_angle_radians = np.deg2rad(wedge_angle)
        self.n_factor = np.pi / wedge_angle_radians
        self.c_factor = c_factor

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the wedge flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z_translated = z - self.translation_x - 1.0j * self.translation_y
        r = np.absolute(z_translated)
        theta = np.arctan2(np.imag(z_translated), np.real(z_translated))
        theta = np.mod(theta, 2.0*np.pi)
        complex_velocity = self.n_factor * self.c_factor * r**(self.n_factor-1.0) * np.exp(1.0j * (self.n_factor-1.0) * theta)
        return complex_velocity


class SourceFlow:

    def __init__(self, source_strength: float, translation_x: float = 0.0, translation_y: float = 0.0) -> None:
        """Initializer for the source- and sink-flow class. Stores relevant quantities as class members.

        Args:
            source_strength (float): The strength of the sourcce flow. It is positive for source flow and negative for sink flow.
            translation_x (float, optional): The translation of the singularity in x direction. Defaults to 0.0.
            translation_y (float, optional): The translation of the singularity in y direction. Defaults to 0.0.
        """
        self.translation_x = translation_x
        self.translation_y = translation_y
        self.source_strength = source_strength
        self.factor = self.source_strength / (2.0 * np.pi)

    def get_complex_velocity(self, z: np.array) -> np.array:
        """Returns the complex velocity of the source-/sink-flow.

        Args:
            z (np.array): Complex numpy array containing the meshgrid of the complex variable z.

        Returns:
            np.array: The complex velocity as a complex numpy array.
        """
        z_translated = z - self.translation_x - 1.0j * self.translation_y
        return self.factor / z_translated
    
def get_grid_quantities(x_min: float, x_max: float, y_min: float, y_max: float, resolution_x: int, resolution_y: int):
    """Returns the meshgrid used for the calculations.

    Args:
        x_min (float): The minimum x value.
        x_max (float): The maximum x value.
        y_min (float): The minimum y value.
        y_max (float): The maximum y value.
        resolution_x (int): The number of points to discretize in x direction.
        resolution_y (int): The number of points to discretize in y direction.

    Returns:
        List[np.array]: The numpy array containing the meshgrid.
    """
    x_linspace = np.linspace(x_min, x_max, resolution_x)
    y_linspace = np.linspace(y_min, y_max, resolution_y)
    X_meshgrid, Y_meshgrid = np.meshgrid(x_linspace, y_linspace)
    Z_meshgrid = X_meshgrid + 1.0j * Y_meshgrid
    return X_meshgrid, Y_meshgrid, Z_meshgrid

## Definition of domain

Here, relevant information to describe the domain is provided. It is used to generate the meshgrid for the calculations.

In [ ]:
x_min = -2.0
x_max = 2.0
y_min = -2.0
y_max = 2.0
resolution_per_unit_length = 50
resolution_x = ( x_max - x_min ) * resolution_per_unit_length
resolution_y = ( y_max - y_min ) * resolution_per_unit_length

X_meshgrid, Y_meshgrid, Z_meshgrid = get_grid_quantities(x_min, x_max, y_min, y_max, int(resolution_x), int(resolution_y))


## Calculation of the complex velocity

Initializes the relevant source, sink and parallel flow objects to calculate the complex velocity.

In [ ]:
v_dot = 1.5
source_strength = 4.0 / 3.0 * v_dot
a = 1.0
u_a = 1.0
wedge_angle = 270  # Wedge angle in degree
c_factor_wedge = (3.0/2.0) * (u_a * a**(1.0/3.0) - source_strength / (2.0 * np.pi * a**(2.0/3.0)) )

source = SourceFlow(source_strength, 0.0, 0.0)
wedge = WedgeFlow(wedge_angle, c_factor_wedge, 0.0, 0.0)

complex_velocity = source.get_complex_velocity(Z_meshgrid) + wedge.get_complex_velocity(Z_meshgrid)
u = np.real(complex_velocity)
v = - np.imag(complex_velocity)

## Plotting of the results

Plot the streamlines and potential lines.
Streamlines are plotted in TUMBlue.
The stagnation streamline is highlighted in red.
Potential lines are plotted in TUMOrange.
The contour of the wedge is plotted in black.

In [ ]:
fig, ax = plt.subplots()

x_linspace = np.linspace(x_min, 0.0, 21)
y_linspace = np.linspace(x_min, x_max, 20)

# Plot the streamlines.

streamline_seeds = np.stack([x_linspace, np.ones_like(x_linspace)*0.0], axis=-1)

ax.streamplot(
    X_meshgrid, Y_meshgrid, u, v,
    density=[1.0, 1.0],
    minlength=1.0, maxlength=20.0,
    broken_streamlines=False,
    start_points=streamline_seeds,
    color=TUMBlue,
)

stagnation_point_x = 0.0
stagnation_point_y = -np.power( ( a**(2.0/3.0) * v_dot ) / ( (3.0/2.0) * np.pi * u_a * a - v_dot ) , 3.0/2.0)
stagnation_streamline_seed = np.array([stagnation_point_x, stagnation_point_y]).reshape([1,2])
print(stagnation_streamline_seed)

ax.streamplot(
    X_meshgrid, Y_meshgrid, u, v,
    minlength=1.0, maxlength=20.0,
    broken_streamlines=False,
    start_points=stagnation_streamline_seed,
    color="red",
    linewidth=3.0,
)

# Plot the potential lines.

ax.streamplot(
    X_meshgrid, Y_meshgrid, -v, u,
    density=[0.2, 0.2],
    minlength=1.0, maxlength=20.0,
    broken_streamlines=False,
    arrowstyle = "-",
    color=TUMOrange,
)

ax.plot([0.0, 0.0], [y_min, 0.0], color="black", linewidth=3.0)
ax.plot([0.0, x_max], [0.0, 0.0], color="black", linewidth=3.0)

ax.set_aspect('equal', 'box')
plt.show()

Plotting streamlines and potential lines without prescribing any seed, and by that not imposing prior knowledge, is also possible

In [ ]:
plt.close()

fig, ax = plt.subplots()

# Plot the streamlines.
ax.streamplot(
    X_meshgrid, Y_meshgrid, u, v,
    density=[1.0, 1.0],
    color=TUMBlue,
)

# Plot the potential lines.
ax.streamplot(
    X_meshgrid, Y_meshgrid, -v, u,
    density=[0.2, 0.2],
    arrowstyle = "-",
    color=TUMOrange,
)

ax.set_aspect('equal', 'box')
plt.show()